# OMS Geofabrik

OpenStreetMap (OSM) is a collaborative project that creates a free, editable map of the world, used widely for geospatial analysis and mapping projects.

Geofabrik is a well-known provider that offers regularly updated regional extracts of OSM data in convenient file formats like .osm.pbf (a compressed binary format). 

`.osm.pbf` file includes geographic features such as: Roads(walking, cycling, driving...), Buildings, Points of interest ( manity, shop, tourism...)

For more details on how to work with this data using `pyrosm`, see the [Pyrosm Basics Documentation](https://pyrosm.readthedocs.io/en/latest/basics.html).

## Setup

Package install from terminal (if required)

```
conda install -c conda-forge pyrosm
```

In [1]:
from pyrosm import OSM, get_data
from pyrosm.data import sources
import geopandas as gpd
import pandas as pd

## Download data 

`Pyrosm` allows to download any PBF dataset available at ***Geofabrik*** or ***BBBike*** to our local machine without the need to go to the website and do this manually. Currently, PBF data can be downloaded from 654 regions in the world.Let's explore which region is available.

### Available dataset

In [2]:
sources.available.keys()

dict_keys(['africa', 'antarctica', 'asia', 'australia_oceania', 'central_america', 'europe', 'north_america', 'south_america', 'cities', 'subregions'])

In [3]:
print(sources.asia.available)

['afghanistan', 'armenia', 'azerbaijan', 'bangladesh', 'bhutan', 'cambodia', 'china', 'east-timor', 'gcc_states', 'india', 'indonesia', 'iran', 'iraq', 'israel_and_palestine', 'japan', 'jordan', 'kazakhstan', 'kyrgyzstan', 'laos', 'lebanon', 'malaysia_singapore_brunei', 'maldives', 'mongolia', 'myanmar', 'nepal', 'north_korea', 'pakistan', 'philippines', 'south_korea', 'sri_lanka', 'syria', 'taiwan', 'tajikistan', 'thailand', 'turkmenistan', 'uzbekistan', 'vietnam', 'yemen']


China is available. 

### Download and refresh the data

In [4]:
print("\nDownloading...")
fp = get_data("China", update=True, directory= "../../data/mapdata") # remove the old OBF file and download a fresh version from Geofabrik
print("Data was downloaded to:", fp)


Downloading...


ValueError: The provided directory does not exist: ../../data/mapdata

In [ ]:
osm = OSM("../../data/mapdata/china-latest.osm.pbf")

 ## Exploratory analysis

In [ ]:
solar_tags = {
    "power": ["plant"],
    "plant:source": ["solar"]
}

solar_pois = osm.get_pois(custom_filter=solar_tags)

print(solar_pois[["name", "power", "plant:source", "geometry"]].head())
print(solar_pois.shape)


MemoryError: Unable to allocate output buffer.

### Save as Parquet

In [ ]:
parquet_path = "../../data/mapdata/china_provinces.parquet"
osm.to_parquet(parquet_path, index=False)
print(f"Saving file as parquet : {parquet_path}")

AttributeError: 'OSM' object has no attribute 'to_parquet'

In [ ]:
#delete OMS file (too big for github)
try:
    os.remove(pbf_path)
    print(f"✅ PBF file deleted: {pbf_path}")
except Exception as e:
    print(f"⚠️ Error when deleting PBFb file: {e}")

⚠️ Error when deleting PBFb file: name 'os' is not defined
